## Load CSV from spotify

In [1]:
import pandas as pd
import numpy as np

In [15]:
df_spotify = pd.read_csv('SpotifyFeatures.csv')

In [16]:
Genre_to_remove = ['Anime', "Children's Music", 'Children’s Music', 'Classical', 'Soundtrack', 'Opera', 'A Capella', 'Movie']
for genre in Genre_to_remove:
    df_spotify = df_spotify[df_spotify.genre != genre]

In [17]:
to_search = df_spotify[['artist_name', 'track_name', 'popularity', 'genre']]

We need to remove all the occurences of feat

In [18]:
def remove_feat(track_name):
    if '(' in track_name:
        track = track_name.split('(')
        return track[0]
    else:
        return track_name
    
def remove_space(track_name):
    if ' - ' in track_name:
        track = track_name.split(' - ')
        return track[0]
    else:
        return track_name

In [19]:
to_search['track_name'] = to_search['track_name'].apply(lambda x: remove_feat(x))

C:\Users\Alexis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
to_search = to_search.drop_duplicates()

In [21]:
to_search['track_name'] = to_search['track_name'].apply(lambda x: remove_space(x))

In [22]:
# save df to pickle
to_search.to_pickle("to_search.pkl")

In [2]:
# read df from pickle
to_search = pd.read_pickle("to_search.pkl")
to_search = to_search.reset_index().drop(['index'],axis=1)

In [12]:
to_search[to_search.track_name == 'For the Widows in Paradise, For the Fatherless in Ypsilanti']

,artist_name,track_name,popularity,genre
101118,Sufjan Stevens,"For the Widows in Paradise, For the Fatherless...",50,Indie


In [13]:
to_search = to_search[to_search.index >= 101118]

In [14]:
to_search

,artist_name,track_name,popularity,genre
101118,Sufjan Stevens,"For the Widows in Paradise, For the Fatherless...",50,Indie
101119,NEEDTOBREATHE,The Outsiders,49,Indie
101120,Moon Taxi,Let The Record Play,45,Indie
101121,The Mayries,High Hopes,62,Indie
101122,Sabrina Claudio,Natural,49,Indie
...,...,...,...,...
173883,Slave,Son Of Slide,39,Soul
173884,Jr Thomas & The Volcanos,Burning Fire,38,Soul
173885,Muddy Waters,,47,Soul
173886,R.LUM.R,With My Words,44,Soul


## Load musics from Genius

In [6]:
import requests
from bs4 import BeautifulSoup


def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + '6F7cdWVweWfmPXo-Nm-weqZ8_ouzvIk_ZfEXpNojIBdimY6ZI_e2zjPQiCcIU6YT'}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html5lib')

    return html

Test by directly create different rows for different writers of the same song

In [7]:
# if already created
artists_writers_df = pd.read_pickle("artists_writers_df.pkl")
artists_writers_df = artists_writers_df.drop_duplicates(['track_name', 'artist'])

In [46]:
artists_writers_df = pd.DataFrame(columns=['track_name', 'artist', 'genre', 'popularity', 'more_infos'])

In [8]:
counter = 0
for row in to_search.iterrows():
    artist = row[1]['artist_name']
    song = row[1]['track_name']
    popularity = row[1]['popularity']
    genre = row[1]['genre']
    
    r = request_song_info(song, artist)
    content = r.json()
    if len(content['response']['hits']) != 0:
        counter +=1
        true_content = content['response']['hits'][0]['result']

        url = true_content['url']
        soup2 = scrap_song_url(url)

        infos = soup2.find_all('meta')[-1]

        print(song + ', ' + artist + ', ' + str(popularity))
        to_add = {'track_name': song, 'artist': artist,\
                  'popularity': popularity, 'genre': genre, \
                  'more_infos':(str(infos).replace('&quot;', '').split('label:')[1:])}
        
        artists_writers_df = artists_writers_df.append(to_add, ignore_index=True)
        
        if counter%1000 ==0:
            # save df to pickle
            print('saving ...')
            artists_writers_df.to_pickle("artists_writers_df.pkl")
            print('saved')

No Sympathy, Peter Tosh, 38
Love Is Lost, Summer Thieves, 47
Close My Eyes, Matisyahu, 36
Drink Professionally, Slightly Stoopid, 31
Never Ever, SOJA, 42
Still the same, The Holdup, 31
Speak Easy, 311, 38
East Point, Rambutan Jam Band, 40
Ever Really Wanted, Slightly Stoopid, 31
Simplicity, Nahko and Medicine for the People, 31
Food For Thought, UB40, 49
Head In My Hands, The Expendables, 33
One More Night, One Drop, 33
My League, Mavado, 44
Visions , Dirty Heads, 31
The Truth, The Holdup, 31
After Another, Spawnbreezie, 40
My Lord, Michael Franti & Spearhead, 33
Planet Junior, Babe Rainbow, 45
Zion Gate Dub, King Tubby, 40
I Ain't Running, Sticky Fingers, 48
Move Like Creatures, Katastro, 33
Perfect, Christopher Martin, 46
Extension, 311, 39
Just Remember, House Of Shem, 43
A Little Bit Of Riddim, Michael Franti & Spearhead, 33
Time Won't Wait, Slightly Stoopid, 31
One True, Mishka, 39
Juicy Ones, Sticky Fingers, 48
Better, SOJA, 38
Expectations, Katastro, 33
No Way No , Los Pericos, 

Come Back Home, Beres Hammond, 35
In The Rain, Ernest Ranglin, 36
Natural Mystic, Bob Marley & The Wailers, 40
Everything You Need, Slightly Stoopid, 31
Like Your Style, Pepper, 30
Lincoln Highway Dub, Sublime, 33
Grass Is Greener, Bumpin Uglies, 31
Pressure, Skindred, 47
Lejos, Black Guayaba, 34
Garden Grove, Sublime, 36
Better People, Xavier Rudd, 45
Love's Contagious, Tarrus Riley, 37
Don't Stay Home, 311, 37
Sensemelia Persecution, Buju Banton, 34
Exodus, Bob Marley & The Wailers, 38
Black Cat, Ziggy Marley, 36
Rivers Of Babylon, Sublime, 31
Intro, Damian Marley, 40
Marijuana, Richie Spice, 39
Wings Of The Morning, Capleton, 32
Beat & Teach, Mavado, 39
Can't Get Me Down, Warrior King, 37
Falling For You, Tribal Theory, 32
Tune In, Gregory Isaacs, 35
Kinky Reggae, Bob Marley & The Wailers, 37
Somos Sur, Ana Tijoux, 52
Fight, Anthony B, 0
Only Woman D.J. With Degree, Sister Nancy, 36
More Than Dub , Rebelution, 30
Astuta, Santaferia, 60
My Love, Fortunate Youth, 32
Keep The Faith, An

Ring the Alarm , Buju Banton, 33
Big Salty Tears, The Ziggens, 32
Mal De Amore, Vicente Garcia, 45
She Wants to Destroy Me, Ballyhoo!, 31
Right Back, Sublime, 35
Por Cultivar Marihuana, Resistencia Suburbana, 52
Bocanada, Cultura Profética, 45
Jellyfish, Ocean Alley, 49
My World, Sublime With Rome, 31
Come & Go, Aer, 36
Stepping Out, Steel Pulse, 36
Love Black Woman, Buju Banton, 33
Hello Bonjour, Michael Franti & Spearhead, 32
Bending , Katastro, 31
Leisure, Israel Vibration, 36
Kingdom Of Zion, Alborosie, 43
Kindness For Weakness, Glen Washington, 38
Rolling Stone , Slightly Stoopid, 32
Soul Rebels, Bob Marley & The Wailers, 38
Fight This Feeling, Shaggy, 39
Can't Hardly Wait, Ballyhoo!, 31
Get Myself To Saturday, Michael Franti & Spearhead, 31
Music, The Beautiful Girls, 30
Music Move Me, The Expendables, 31
Safe And Sound, Sublime With Rome, 30
Lluvia, Nubes, Tristeza, Dread Mar I, 51
Still Standing, Michael Franti & Spearhead, 31
Shade from the Sun, Matisyahu, 35
The Better Half O

Am-A-Do, Bob Marley & The Wailers, 35
Running Back, Pacific Dub, 31
Unbreakable , Alborosie, 42
So True, The Black Seeds, 32
Bowl for Two , The Expendables, 32
Nothing I Wouldn't Do, The Expendables, 31
Dear Congress, , Dispatch, 36
Tomorrow's Another Day, Collie Buddz, 33
Square Circles , Matisyahu, 33
Invasion , Rebelution, 30
Do It For The Love, Michael Franti & Spearhead, 30
Exodus, Bob Marley & The Wailers, 38
Jahwakening, Mike Love, 37
I Got Love For Ya, Michael Franti & Spearhead, 31
Exaltation, Matisyahu, 34
So High, Arise Roots, 30
World Revolution, Ziggy Marley, 29
Love Is, Jah Cure, 38
Sweet Fanta Diallo, Alpha Blondy, 29
Burn Dem Down, Capleton, 43
Hello Africa, Garnett Silk, 35
Complaint, Buju Banton, 33
Core, MAGIC!, 41
Fine Day Together, SDIB, 31
Hot Cookin', G. Love & Special Sauce, 30
My Generation, Nas & Damian "Jr. Gong" Marley, 29
Shadow, SOJA, 33
Nuh Linga, Elephant Man, 28
Tear It Down, SOJA, 40
Engine Trouble, 10 Ft. Ganja Plant, 32
Dem Gone, Gentleman, 37
Just W

Taxi Driver, Steel Pulse, 34
New Love, Kash'd Out, 28
Talking Blues, The Gladiators, 38
Let Love In, Dennis Brown, 37
Shalom Salaam, Ziggy Marley, 35
Freedom Fighter, Anthony B, 46
Adelphia, The Cat Empire, 45
Undivided, Groundation, 38
For Real, Natural Incense, 29
Come Down, Collie Buddz, 34
Natty Dread A Weh She Want, Horace Andy, 37
What You Like, Pacific Dub, 30
Si No Te Tengo, Green Valley, 55
In With the Newport, Wheeland Brothers, 29
Awake, The Hip Abduction, 28
For The Love Of You, John Holt, 38
Do It Twice, The Wailers, 33
A Pocketful Of Shells, Mat McHugh, 42
Inspiration, E.N Young, 30
Personalmente, Las Pelotas, 64
Street Corner, The Skatalites, 46
Your Best Friend, Morgan Heritage, 36
Letter to Lady J, Dispatch, 34
Sun Is Shining, Bob Marley & The Wailers, 35
WHY MUST I CRY, Peter Tosh, 33
I Don't Wanna Live on Mars, Ziggy Marley, 28
Walk Tall, Ziggy Marley, 32
Let Her Dance, Stephen Marley, 31
Home, Iration, 29
Nada De Ti, Dread Mar I, 50
Pimp Floyd, Desmond Cheese, 40
Vi

...

The Dance, Madaila, 44
Blackout, Freya Ridings, 35
Beautiful, Ben Rector, 50
Stay a Little Longer, Lostboycrow, 48
May I Have This Dance, Meadowlark, 47
Territory, The Blaze, 64
Fakin, Hoodie Allen, 48
Fever, Roosevelt, 51
Underdog, Kasabian, 63
Spitting Venom, Modest Mouse, 47
Aging out of the 20th Century, Trash Panda, 45
Bonfire, The Hunna, 47
Wild World, Joanna Wang, 61
Tell Me Where It Hurts, MYMP, 67
salvia plath, Teen Suicide, 47
Ladada, Dr. Dog, 48
Again, LAOISE, 56
Pansy Waltz, Shakey Graves, 47
Sa Dulo Ng Walang Hanggan, December Avenue, 68
As Much as I Ever Could, City and Colour, 44
Slop, Forth Wanderers, 44
Bedouin Dress, Fleet Foxes, 45
Give It Up, Angel Olsen, 50
Kim, Joy Again, 45
Girl Of The Year, Beach House, 46
Bloodflows, SOHN, 52
Terrible Love, The National, 51
Isolation, Cherry Glazerr, 45
Green, Ben Rector, 50
Piece of Mind 3, Witt Lowry, 49
Do Not Disturb, Teyana Taylor, 54
A Plus, Zak Downtown, 50
Letter By The Water, The Japanese House, 43
You Love Me Anyway, 

Lookers, The Menzingers, 49
GOAT, Ari Lennox, 52
Tricks, Stella Donnelly, 48
Game, Mating Ritual, 49
Cold Is The Night, The Oh Hellos, 48
The Warning, Hot Chip, 58
Sweet Love, Phillip Larue, 48
Like The Dawn, The Oh Hellos, 50
sublime, slenderbodies, 47
Salvation, Gabrielle Aplin, 53
I Found, Amber Run, 54
Running Scared , The Strumbellas, 46
Cuffed, Nick Hakim, 45
Fuckin 'n' Rollin, Phantastic Ferniture, 44
Finger, Ty Segall, 51
Lullaby, Sleeping At Last, 52
Let's Get Married, Vedo, 49
Ayahuasca, Vancouver Sleep Clinic, 51
Timeless, James Blake, 50
Westside, The Kooks, 52
Nice And Slow, Max Frost, 46
I Know A Place, MUNA, 50
Planet Telex, Radiohead, 57
Your Love, Tom Misch, 54
Be the Song, Foy Vance, 47
Sleepless In NYC, Autograf, 51
Crater Lake, Lady Lamb, 49
Savior, St. Vincent, 50
Family Tree, Kings of Leon, 55
Solitaire, MARINA, 47
Candy Eyes, Jack Stauber, 45
John Beyers , Colter Wall, 47
Things It Would Have Been Helpful to Know Before the Revolution, Father John Misty, 47
Black

what about today?, Lewis Watson, 48
Alright, Keaton Henson, 44
The Dress Looks Nice on You, Sufjan Stevens, 49
After The Afterparty , Charli XCX, 61
Amor Eterno, Silvana Estrada, 57
INSIDE OUT, Luke Christopher, 49
Fucked Up, Young Rising Sons, 44
Late Bloomers, The Growlers, 46
Bring My Baby Back, Dr. Dog, 48
I Got You, Sofia Karlberg, 50
Crossfire, Brandon Flowers, 56
Closer, Nick Wilson, 45
There's A Reason Why , Blossoms, 66
I Just Know, Bugus, 52
Bruise Cruise, Puma Blue, 47
Shattered & Hollow, First Aid Kit, 50
Timid, Jeebanoff, 50
Alison, Swimming Tapes, 45
Love Is Overrated, Shwayze, 47
Alabama Pines, Jason Isbell and the 400 Unit, 52
Innocent, Eric Bellinger, 52
Home at Last, HOMESHAKE, 46
Not The Man, The Growlers, 45
Jailbreak, AWOLNATION, 52
Eroplanong Papel, December Avenue, 65
Body Sun, RY X, 56
Morning Light, Josh Garrels, 47
Wake Me, Mandolin Orange, 50
Chicago, Christian Leave, 46
Summer Days, A R I Z O N A, 54
Human, Sevdaliza, 55
Bury Me With It, Modest Mouse, 46
She

This Too Shall Pass, OK Go, 44
Gonna Hurry , Whitney, 49
Drama 2.0, NoMBe, 31
Range Life , Pavement, 51
Hurt Somebody , Noah Kahan, 57
Bluebird, Beach House, 44
One Of THOSE Nights, The Cab, 51
On Your Own, Justin Stone, 53
Animal, Miike Snow, 46
Powder, Gregory Alan Isakov, 49
Bros, Wolf Alice, 47
Styrofoam Boots/It's All Nice on Ice, Alright, Modest Mouse, 45
I Need A Woman, Hockey Dad, 43
Careful You, TV On The Radio, 49
Can It All Be So Simple, El Michels Affair, 49
How to Love , Cash Cash, 50
PURPLE TUESDAY , Rejjie Snow, 50
You're Still a Mystery, Bleachers, 47
The Donor Blues, Shakey Graves, 47
True Colours, Tom Odell, 62
Fingerprints, Hiatus Kaiyote, 54
Why Can't I?, Liz Phair, 54
Problems, Pinegrove, 44
saving ...
saved
True Care, James Vincent McMorrow, 44
Silence, Grace Carter, 57
Ace In The Hole, Saint Motel, 45
I Don't Even Smoke Weed, Empress Of, 52
Sailboat, Ben Rector, 49
Never Gone, Said the Sky, 48
Glass in the park, Alex Turner, 45
Problems, anders, 56
Slug, Snail Ma

KeyboardInterrupt: 

In [9]:
# save df to pickle
artists_writers_df.to_pickle("artists_writers_df.pkl")

In [52]:
artists_writers_df = artists_writers_df.reset_index().drop(['index'], axis=1)

In [10]:
artists_writers_df

,track_name,artist,genre,popularity,more_infos
0,Be Without You,Mary J. Blige,R&B,65,"[Strings,artists:[{_type:artist,api_path:/arti..."
1,Desperado,Rihanna,R&B,63,"[Distributor,artists:[{_type:artist,api_path:/..."
2,Ice On My Baby,Yung Bleu,R&B,62,"[Video,overlay_compatible_image_url:https://im..."
3,Heaven Falls / Fall on Me,Surfaces,R&B,61,[]
4,Love Myself,Olivia O'Brien,R&B,68,"[Video,overlay_compatible_image_url:https://im..."
...,...,...,...,...,...
74268,Snake Mountain Blues,Colter Wall,Indie,45,"[Acoustic Guitar,artists:[{_type:artist,api_pa..."
74269,Everywhere I Go,New Politics,Indie,47,[]
74270,Like I Did,JC Stewart,Indie,55,[]
74271,Thank You,MKTO,Indie,53,[]
